#Importo librerie

In [1]:
import numpy as np
import pandas as pd

from time import time

#Autenticazione Spotify API

In [2]:
!pip uninstall spotipy

In [3]:
!pip install spotipy

     |████████████████████████████████| 153kB 11.8MB/s 
     |████████████████████████████████| 61kB 5.7MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [4]:
# autenticazione Spotify API con spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
auth_manager = SpotifyClientCredentials(client_id='caf57b996b464996bff50ab59186f265', client_secret='0bfcdbff8015426cae855b56b692f69b')
sp = spotipy.Spotify(auth_manager=auth_manager, requests_timeout=60)

#Importo dataset

In [5]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [6]:
drive.CreateFile({'id':'1PmNTXFn5c63BhxarokbWLs2FxdxljhDF'}).GetContentFile('dataset2_X_billboard_plus.csv')  # NB: dataset prima dell'estrazione delle new_features
df = pd.read_csv("dataset2_X_billboard_plus.csv").drop('Unnamed: 0',axis=1)

In [7]:
from google.colab import  drive

# mounts the google drive to Colab Notebook
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


#Definizione funzioni


In [8]:
def fix_artists_col(row):
  print('\nprocessing id:%s...' % row.id)

  artists_data = sp.track(row.id)['artists']
  
  artists_array = []

  for i in range(len(artists_data)):
    artists_array.append(artists_data[i]['name'])

  return artists_array

In [9]:
def print_exec_time(start):
  print("\nEsecuzione completata in %.2f secondi" % (time()-start))

In [45]:
def get_past_popularity(row):
  print('\nprocessing id:%s...' % row.id)

  n_hits = 0 
  n_weeks = 0 

  for artist in row.artists:
    sub_df = df[df.artists.apply(lambda x: artist in x)]  # filtro in base ad artista
    sub_df = sub_df[sub_df.release_date < row.release_date] # filtro in base a release_date (seleziono canzoni precedenti) 
    
    n_hits += sub_df.hit.sum()  # conto hit --> sommo al totale
    n_weeks += sub_df.weeks.sum()  # conto weeks --> sommo al totale
    
  return n_hits, n_weeks

#Dataset fix


In [22]:
df[df.artists.str[0] != '['].shape

(11666, 21)

In [23]:
df_to_fix = df.iloc[-11666:]  # datapoints aggiunti --> da modificare
df_ok = df.iloc[:-11666]  # datapoints ok

In [24]:
df_to_fix.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,hit,weeks
111546,0.584,2021,0.18100,Justin Bieber,0.686,190779.0,0.538,0,31qCy5ZaophVA81wtlwLc4,0.000003,2.0,0.1130,-8.026,1.0,Anyone,0.0,2021-01-01,0.0345,115.884,1.0,1.0
111547,0.915,2020,0.00883,Dua Lipa Featuring DaBaby,0.702,203064.0,0.825,0,463CkQjx2Zk1yXoBuierM9,0.000000,6.0,0.0674,-3.787,0.0,Levitating,0.0,2020-10-01,0.0601,102.977,1.0,14.0
111548,0.485,2020,0.23700,Ariana Grande,0.830,173711.0,0.585,1,6Im9k8u9iIzKMrmV7BWtlF,0.000000,0.0,0.2480,-6.476,1.0,34+35,0.0,2020-10-30,0.0940,109.978,1.0,10.0
111549,0.457,2020,0.00786,Ava Max,0.637,162399.0,0.690,0,7a53HqqArd4b9NF4XAmlbI,0.000000,1.0,0.1240,-4.057,0.0,Kings & Queens,0.0,2020-09-18,0.0405,129.857,1.0,22.0
111550,0.469,2020,0.01660,Megan Thee Stallion,0.914,171067.0,0.732,1,0A1hoCfMLkiAgvhWkkucJa,0.000000,8.0,0.1070,-6.100,0.0,Body,0.0,2020-11-20,0.2410,94.066,1.0,7.0


In [25]:
time_0 = time()
df_to_fix.artists = df_to_fix.apply(fix_artists_col, axis=1)
print_exec_time(time_0)

Output streaming troncato alle ultime 5000 righe.

processing id:4URcCVFgWv9cWZxgjhveFj...

processing id:7xpfik1yecW50767Bfx8lP...

processing id:0nc6Xu6bjDZz8UTyU5WHBS...

processing id:3u7n1FGFhlUEZrjCmwwepe...

processing id:3g9731HiHbDvSwxVf7R2CZ...

processing id:6cGWnETSKdUw8JU8MxKv5W...

processing id:5O0ibxlu6WLiDr7Q2n8EuI...

processing id:3EQM6RIkUZ3nVI8mTVwIvY...

processing id:49CnPV5v4T9AX2rubSK5S1...

processing id:56aCFmwTdm1me0suarfiWn...

processing id:19jo0UT2vqD4pNVfIqTy4R...

processing id:7EiWrR27y0mfzEJrsoSwQU...

processing id:59HTJznkQkbBnfpe9KhIRJ...

processing id:0RZbhSQw2Z7NeOgzi5Gt37...

processing id:1B0YNcxRYQ4vJIcFFJRCt8...

processing id:0kYqTzbLCdlEdAL3snnZHM...

processing id:3H10ZvoLhe5sHpTu3slqlr...

processing id:4fSQBnovCFWkLhMbWkmsWc...

processing id:6Y93DEf2OkUgksFCGHr8Zx...

processing id:44okWRzv99N7MzyYiC4SRU...

processing id:1FRVMsTQHwcDhZBBDKfhyT...

processing id:2GEW7EFcnqHqugI6KOSPm8...

processing id:5Qm5fwmi7kq1bIVYFcjDap...

proces

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [26]:
# converto colonna artists da stringa a lista di stringhe
df_ok.artists = df_ok.artists.apply(eval)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [27]:
df = pd.concat([df_ok, df_to_fix])  

In [28]:
# converto colonna 'release_date' in tipo datetime
df.release_date = pd.to_datetime(df.release_date,format="%Y-%m-%d",exact=False)

#past_populary

In [72]:
# aggiungo campi: 'past_pop_n_hit' --> intero che rappresenta il numero di canzoni che l'artista ha avuto in classifica prima della canzone in questione
#                 'past_pop_n_weeks' --> intero che rappresenta il numero di settimane complessive in cui l'artista ha avuto canzoni in classifica prima della canzone in questione

time_0 = time()

df[['past_pop_n_hit', 'past_pop_n_weeks']] = df.apply(get_past_popularity, axis=1, result_type='expand')

print_exec_time(time_0)

Output streaming troncato alle ultime 5000 righe.

processing id:4URcCVFgWv9cWZxgjhveFj...

processing id:7xpfik1yecW50767Bfx8lP...

processing id:0nc6Xu6bjDZz8UTyU5WHBS...

processing id:3u7n1FGFhlUEZrjCmwwepe...

processing id:3g9731HiHbDvSwxVf7R2CZ...

processing id:6cGWnETSKdUw8JU8MxKv5W...

processing id:5O0ibxlu6WLiDr7Q2n8EuI...

processing id:3EQM6RIkUZ3nVI8mTVwIvY...

processing id:49CnPV5v4T9AX2rubSK5S1...

processing id:56aCFmwTdm1me0suarfiWn...

processing id:19jo0UT2vqD4pNVfIqTy4R...

processing id:7EiWrR27y0mfzEJrsoSwQU...

processing id:59HTJznkQkbBnfpe9KhIRJ...

processing id:0RZbhSQw2Z7NeOgzi5Gt37...

processing id:1B0YNcxRYQ4vJIcFFJRCt8...

processing id:0kYqTzbLCdlEdAL3snnZHM...

processing id:3H10ZvoLhe5sHpTu3slqlr...

processing id:4fSQBnovCFWkLhMbWkmsWc...

processing id:6Y93DEf2OkUgksFCGHr8Zx...

processing id:44okWRzv99N7MzyYiC4SRU...

processing id:1FRVMsTQHwcDhZBBDKfhyT...

processing id:2GEW7EFcnqHqugI6KOSPm8...

processing id:5Qm5fwmi7kq1bIVYFcjDap...

proces

In [73]:
df.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,hit,weeks,past_pop_n_hit,past_pop_n_weeks
0,0.207,2020,0.9170,[Sam Smith],0.758,220237.0,0.155,0,74Rr0afCO2T2v0Xmaq7Shz,0.000000,3.0,0.116,-10.252,1.0,Forgive Myself,5.0,2020-11-24,0.0447,122.026,0.0,0.0,17.0,336.0
1,0.302,2020,0.8050,[Sam Smith],0.345,257651.0,0.340,0,0fGnjXBhmNfWGbL50VhkMd,0.000017,1.0,0.125,-9.337,1.0,Fix You - Live,4.0,2020-11-24,0.0342,136.219,0.0,0.0,17.0,336.0
2,0.497,2020,0.2470,"[DaBaby, Roddy Ricch]",0.746,181733.0,0.690,1,18cHFfnBjVpcQvy9eapiyU,0.000000,11.0,0.101,-7.956,1.0,ROCKSTAR,1.0,2020-11-23,0.1640,89.979,1.0,37.0,39.0,295.0
3,0.548,2020,0.2220,[Arijit Singh],0.654,210620.0,0.713,0,6zgPxJ3HlrD8Vqv7OVLG0Y,0.000000,0.0,0.312,-7.566,1.0,"Nacchite Ye Panaina (From ""Dohchay"")",0.0,2020-11-23,0.0533,134.985,0.0,0.0,0.0,0.0
4,0.702,2020,0.0382,"[DaBaby, Meek Mill]",0.703,178949.0,0.555,1,3y3lDeOb2c3FYCniav00EY,0.000000,8.0,0.246,-8.199,1.0,8 Figures (feat. Meek Mill),63.0,2020-11-20,0.3820,177.205,0.0,0.0,59.0,230.0


#Export


In [74]:
df.to_csv('/content/drive/My Drive/Colab Notebooks/datasets/dataset2_X_billboard_popularity_2.0.csv')